### Question 1 
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

In [1]:
import numpy as np
import pandas as pd

In [2]:
energy=pd.read_excel('Energy Indicators.xls',skiprows=10,usecols=[2,3,4,5],converters={1: (lambda x:x*1000000)},names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'],na_values=np.nan,skipfooter=38)

In [3]:
energy['Country'].replace(regex=True,inplace=True,to_replace=r'\d',value=r'') #matches decimal digits and 0-9
energy['Country'].replace(regex=True,inplace=True, to_replace=r'\(.*?\)',value=r'') 
energy['Country']=energy['Country'].str.strip()

In [4]:
energy.Country.replace(to_replace={"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"},inplace=True)

In [5]:
GDP=pd.read_csv('world_bank.csv',skiprows=4,usecols=range(0,60))
GDP['Country Name'].replace(to_replace={"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"},inplace=True)

In [6]:
ScimEn=pd.read_excel('scimagojr-3.xlsx')

In [7]:
i=[] #because years are stored as string (I know this is more tedious way to do it!)
for x in range(1960,2006):
    i.append(str(x))
GDP.drop(columns=i,axis=1,inplace=True)

In [8]:
ScimEn.drop(ScimEn.index[15:],inplace=True)
ScimEn

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57
5,6,Canada,17899,17620,215003,40930,12.01,149
6,7,Germany,17027,16831,140566,27426,8.26,126
7,8,India,15005,14841,128763,37209,8.58,115
8,9,France,13153,12973,130632,28601,9.93,114
9,10,South Korea,11983,11923,114675,22595,9.57,104


In [9]:
new=pd.merge(pd.merge(ScimEn,energy,on='Country'),GDP,left_on='Country',right_on='Country Name')
new.set_index('Country',inplace=True)

### Question 3
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

In [10]:
j=[]
for x in range(2006,2016):
    j.append(str(x))
new.iloc[:][j].mean(axis=1)

Country
China                 6.857260e+12
United States         1.560666e+13
Japan                 4.959066e+12
United Kingdom        2.684590e+12
Russian Federation    1.648758e+12
Canada                1.609883e+12
Germany               3.529653e+12
India                 1.610576e+12
France                2.685572e+12
South Korea           1.165321e+12
Italy                 2.127937e+12
Spain                 1.408716e+12
Iran                  4.418080e+11
Australia             1.200887e+12
Brazil                1.980922e+12
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

In [11]:
x=new.iloc[:][j].mean(axis=1).nlargest(6).iloc[[-1]].index
((new.loc[x]['2015']/new.loc[x]['2006'])-1)*100

Country
United Kingdom    10.072272
dtype: float64

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?


In [12]:
new['Energy Supply per Capita'].mean()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

In [13]:
x=new[new['% Renewable']==new['% Renewable'].max()]
print(x.index,x['% Renewable'])

Index(['Brazil'], dtype='object', name='Country') Country
Brazil    69.648
Name: % Renewable, dtype: object


### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

In [14]:
new['Ratio']=new['Self-citations']/new['Citations']
y=new.loc[new['Ratio'].idxmax()]
print (y['Country Name'],y['Ratio'])

China 0.6893126179389422


### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?


In [15]:
new['Population']=(new['Energy Supply']/new['Energy Supply per Capita'])
new['Population']=pd.to_numeric(new['Population'])
new['Population'].idxmax()

'China'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

In [16]:
new['Citable documents per person']=new['Citable documents']/new['Population']
np.correlate(new['Citable documents per person'],(new['Energy Supply per Capita']))

array([0.6091199998548664], dtype=object)

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [17]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
Csize=new['Population'].groupby(ContinentDict).size()
Csum= new['Population'].groupby(ContinentDict).sum()
Cmean=new['Population'].groupby(ContinentDict).mean()
Cstd=new['Population'].groupby(ContinentDict).std()

In [18]:
Continent=pd.concat([Csize,Csum,Cmean,Cstd],axis=1)
Continent.columns=['Size', 'Sum', 'Mean', 'Std']

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

In [19]:
#The question is a little unclear. This is from my understanding
print(np.histogram(new['% Renewable'],bins=5)[0])
print (new['% Renewable'].groupby(ContinentDict).size())

[7 4 2 0 2]
Asia             5
Australia        1
Europe           6
North America    2
South America    1
Name: % Renewable, dtype: int64


### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [20]:
PopEst=pd.Series(new['Population'].apply(lambda x: '{:,}'.format(x)))
PopEst

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: Population, dtype: object